In [ ]:
from helper import load_mistral_api_key
import json
import pandas as pd
api_key = load_mistral_api_key(ret_key=True)

In [ ]:
data = {
    "transaction_id": ["T1001", "T1002", "T1003", "T1004", "T1005"],
    "customer_id": ["C001", "C002", "C003", "C002", "C001"],
    "payment_amount": [125.50, 89.99, 120.00, 54.30, 210.20],
    "payment_date": [
        "2021-10-05",
        "2021-10-06",
        "2021-10-07",
        "2021-10-05",
        "2021-10-08",
    ],
    "payment_status": ["Paid", "Unpaid", "Paid", "Paid", "Pending"],
}
df = pd.DataFrame(data)
df

# Using tools
- You can define all tools that you might want the model to call.

In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"status": df[df.transaction_id ==
                          transaction_id].payment_status.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [ ]:
status = retrieve_payment_status(df, transaction_id="T1001")
print(status)

In [ ]:
type(status)

In [ ]:
def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
        return json.dumps(
            {"date": df[df.transaction_id ==
                        transaction_id].payment_date.item()}
        )
    return json.dumps({"error": "transaction id not found."})

In [ ]:
date = retrieve_payment_date(df, transaction_id="T1002")
print(date)

In [ ]:
tool_payment_status = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_status",
        "description": "Get payment status of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [ ]:
tool_payment_date = {
    "type": "function",
    "function": {
        "name": "retrieve_payment_date",
        "description": "Get payment date of a transaction",
        "parameters": {
            "type": "object",
            "properties": {
                "transaction_id": {
                    "type": "string",
                    "description": "The transaction id.",
                }
            },
            "required": ["transaction_id"],
        },
    },
}

In [ ]:
tools = [tool_payment_status, tool_payment_date]
tools

In [ ]:
import functools

names_to_functions = {
    "retrieve_payment_status": functools.partial(retrieve_payment_status, df=df),
    "retrieve_payment_date": functools.partial(retrieve_payment_date, df=df),
}

In [ ]:
names_to_functions["retrieve_payment_status"](transaction_id="T1001")

In [ ]:
from mistralai.models.chat_completion import ChatMessage
from mistralai.client import MistralClient
import os

client = MistralClient(api_key=os.getenv("MISTRAL_API_KEY"))
model = "mistral-large-latest"

chat_history = [
    ChatMessage(role="user", content="What's the status of my transaction?")
]

# Start a chat where the assistant has access to tools
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response.choices[0].message.content

In [ ]:

# save the assistant's response to the chat
chat_history.append(
    ChatMessage(role="assistant", content=response.choices[0].message.content)
)

# add the user's response to the chat
chat_history.append(ChatMessage(
    role="user", content="My transaction ID is T1001."))

chat_history

In [ ]:
# Add the chat history to the chat
response = client.chat(
    model=model, messages=chat_history, tools=tools, tool_choice="auto"
)

response

In [ ]:
response.choices[0].message

In [ ]:
# Add the tool call as an assistant message in the chat history
chat_history.append(response.choices[0].message)

In [ ]:
chat_history

In [ ]:
# Execute the function to obtain tool results
tool_function = response.choices[0].message.tool_calls[0].function
print("tool_function.name", tool_function.name)
print("tool_function.arguments", tool_function.arguments)

args = json.loads(tool_function.arguments)
print("args", args)

function_result = names_to_functions[tool_function.name](**args)
print("function result", function_result)

In [ ]:
# Add the tool result as an tool message in the chat history
tool_msg = ChatMessage(
    role="tool", name=tool_function.name, content=function_result)
chat_history.append(tool_msg)

chat_history

In [ ]:
# Generate the final answer based on the information provided by the tool :
response = client.chat(model=model, messages=chat_history)
response.choices[0].message.content